# Lesson 4: Metropolis‑Hastings

This notebook demonstrates a **Random‑Walk Metropolis–Hastings** sampler for a Bayesian
model with a Normal likelihood (known variance) and a heavy‑tailed *t* prior on the mean `μ`.
The data are the ten percentage changes in company personnel used in the lecture.


## 1  Setup

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import arviz as az
from scipy.stats import t


In [ ]:

# Data from the lesson
y = np.array([1.2, 1.4, -0.5, 0.3, 0.9, 2.3, 1.0, 0.1, 1.3, 1.9])
n = len(y)
ybar = y.mean()

print(f"n = {n}, ȳ = {ybar:.3f}")


## 2  Target density (up to proportionality)
The posterior kernel is
$g(\mu) \;\propto\; \exp\bigl\{ n\bigl(y\_\bar{}\,\mu - \mu^2/2\bigr)\bigr\}\bigl(1+\mu^2\bigr)^{-1}$.

In [ ]:

def log_g(mu, n, ybar):
    """Log of g(μ) (unnormalised posterior)."""
    mu2 = mu ** 2
    return n * (ybar * mu - mu2 / 2.0) - np.log1p(mu2)


## 3  Random‑Walk Metropolis–Hastings sampler

In [ ]:

def mh(n, ybar, n_iter=1000, mu_init=0.0, cand_sd=0.9):
    mu = np.empty(n_iter)
    mu_now = mu_init
    lg_now = log_g(mu_now, n, ybar)
    accpt = 0

    for i in range(n_iter):
        mu_cand = np.random.normal(mu_now, cand_sd)
        lg_cand = log_g(mu_cand, n, ybar)
        alpha = np.exp(lg_cand - lg_now)
        if np.random.rand() < min(1.0, alpha):
            mu_now = mu_cand
            lg_now = lg_cand
            accpt += 1
        mu[i] = mu_now
    return {"mu": mu, "accpt": accpt / n_iter}


### 3.1  Tuning the proposal standard deviation

In [ ]:

np.random.seed(43)
params = [3.0, 0.05, 0.9]
posts = []
for sd in params:
    post = mh(n, ybar, n_iter=1000, mu_init=0.0, cand_sd=sd)
    posts.append(post)
    print(f"cand_sd={sd:<4}  acceptance={post['accpt']:.3f}")

fig, axes = plt.subplots(len(params), 1, figsize=(6, 6), sharex=True)
for ax, post, sd in zip(axes, posts, params):
    ax.plot(post['mu'], lw=0.6)
    ax.set_title(f'cand_sd={sd}')
    ax.set_ylabel('μ')
axes[-1].set_xlabel('Iteration')
fig.suptitle('Trace plots for different proposal scales', y=1.02)
plt.tight_layout()


### 3.2  Starting far from the posterior

In [ ]:

post_far = mh(n, ybar, n_iter=1000, mu_init=30.0, cand_sd=0.9)
print(f"acceptance (far start) = {post_far['accpt']:.3f}")

plt.figure(figsize=(6,2.5))
plt.plot(post_far['mu'], lw=0.6)
plt.xlabel('Iteration'); plt.ylabel('μ')
plt.title('Trace plot (start at μ=30)')
plt.show()


## 4  Posterior vs prior

In [ ]:

burn = 100
mu_keep = post_far['mu'][burn:]

xs = np.linspace(-1.0, 3.0, 400)
plt.figure(figsize=(6,3))
plt.hist(mu_keep, bins=30, density=True, alpha=0.3, label='Posterior (hist)')
from scipy.stats import gaussian_kde
kde = gaussian_kde(mu_keep, bw_method=0.2)
plt.plot(xs, kde(xs), label='Posterior KDE')
plt.plot(xs, t.pdf(xs, df=1), 'k--', label='t prior (df=1)')
plt.axvline(ybar, color='k', lw=1, label='Sample mean')
plt.xlim(-1,3); plt.xlabel('μ'); plt.legend()
plt.title('Posterior vs prior')
plt.show()
